In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import inspect
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

In [2]:
# Load data from "maindata.csv" into a DataFrame
maindata_df = pd.read_csv("../Resources/final_sorted_df.csv")

In [3]:
maindata_df = maindata_df.drop(maindata_df.columns[0], axis=1)

In [4]:
maindata_df

,Country,2015,2016,2017,2018,2019,Booster doses per 100 people,Total vaccine doses administered per 100 population,Total Confirmed Cases,Newly Confirmed Cases,Total Deaths,New Deaths,Total Recovered Cases,Newly Recovered Cases,Lat,Lon
0,Afghanistan,10.1,11.8,12.6,14.2,14.8,6.361,56.733,55696,16,2442,4,49281,195,33.000000,65.000000
1,Albania,6.5,6.7,6.5,6.7,NaN,14.095,107.300,104313,986,1736,21,67158,849,41.153300,20.168300
2,Algeria,7.0,6.6,6.3,6.2,6.3,1.313,34.817,112622,161,2973,3,77683,146,28.033900,1.659600
3,Andorra,7.0,7.0,7.2,7.4,7.4,54.497,203.300,10799,24,110,0,10356,37,42.506300,1.521800
4,Angola,2.6,2.7,2.8,2.6,2.7,9.085,84.095,20695,55,502,1,19238,17,-11.202692,17.873887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,Uruguay,8.3,8.7,8.8,9.0,9.4,59.877,259.755,55695,821,595,4,48496,461,-32.522779,-55.765835
167,Uzbekistan,4.7,4.7,4.8,5.1,5.4,47.684,245.600,79804,31,622,0,78333,43,41.377491,64.585262
168,Vanuatu,4.3,2.9,2.8,3.3,3.3,5.390,115.721,1,0,0,0,1,0,-15.376706,166.959158
169,Zambia,4.4,4.5,4.4,5.1,5.3,7.695,74.063,77171,687,1059,8,70800,997,-15.416700,28.283300


In [5]:
# Create an SQLite database engine for the main database
engine = create_engine(f"sqlite:///output/countiesdata.sqlite")
# Create a session to work with the database
session = Session(bind=engine)
# Create a session to work with the database
session = Session(bind=engine)
# Define a class for the country data
Base = declarative_base()

C:\Users\mh30f\AppData\Local\Temp\ipykernel_15124\373165384.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [6]:
class CountryData(Base):
    __tablename__ = 'country_data'
    id = Column(Integer, primary_key=True)
    country_name = Column(String(255))
    gdp_2015 = Column(Float)
    gdp_2016 = Column(Float)
    gdp_2017 = Column(Float)
    gdp_2018 = Column(Float)
    gdp_2019 = Column(Float)
    booster_doses_per_100people = Column(Float)
    total_vaccine_doses_administered_per_100population = Column(Float)
    total_confirmed_cases = Column(Integer)
    newly_confirmed_cases = Column(Integer)
    total_deaths = Column(Integer)
    new_deaths = Column(Integer)
    total_recovered_cases = Column(Integer)
    newly_recovered_cases = Column(Integer)
    lat = Column(Float)
    lon = Column(Float)

In [7]:
# Create the table if it doesn't exist
Base.metadata.create_all(engine)

In [8]:
# Iterate through the rows in the DataFrame
for index, row in maindata_df.iterrows():
    country = row['Country']

    # Create a new instance of the CountryData class for the current country
    country_data_instance = CountryData(
        country_name=country,
        gdp_2015=row['2015'],
        gdp_2016=row['2016'],
        gdp_2017=row['2017'],
        gdp_2018=row['2018'],
        gdp_2019=row['2019'],
        booster_doses_per_100people=row['Booster doses per 100 people'],
        total_vaccine_doses_administered_per_100population=row['Total vaccine doses administered per 100 population'],
        total_confirmed_cases=row['Total Confirmed Cases'],
        newly_confirmed_cases=row['Newly Confirmed Cases'],
        total_deaths=row['Total Deaths'],
        new_deaths=row['New Deaths'],
        total_recovered_cases=row['Total Recovered Cases'],
        newly_recovered_cases=row['Newly Recovered Cases'],
        lat=row['Lat'],
        lon=row['Lon']
    )

    # Add the instance to the main database session
    session.add(country_data_instance)

In [9]:
# Commit the changes to the main database
session.commit()

In [10]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)
# Collect the names of tables within the database
inspector.get_table_names()

['country_data']

In [11]:
# Close the session
session.close()